In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Type_1_Diabetes_Risk/GSE30209'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide expression kinetics of children with T1D-associated autoantibodies compared to healthy matched controls II"
!Series_summary	"To unravel genes and molecular pathways involved in the pathogenesis of type 1 diabetes (T1D), we performed genome-wide gene expression profiling of prospective venous blood samples from children developing T1D-associated autoantibodies or progressing towards clinical diagnosis."
!Series_overall_design	"58 peripheral blood RNA samples from 4 autoantibody-positive children and their matched controls were analyzed with Illumina Sentrix WG-6 v2 genome-wide arrays, in order to study the gene expression changes occuring during the pathogenesis of Type 1 diabetes (T1D). Each case child (positive for T1D-specific autoantibodies) was matched with a persistently autoantibody-negative control child, with the same HLA-DQB1 risk category, gender, and place and date of birth. Seroconversion is determined as the first detectio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Determine gene expression data availability
is_gene_available = True

# Variable availability and data type conversion

# Type_1_Diabetes_Risk: inferred from seroconversion or diagnosis data
# Age: available as age at sample (months)
# Gender: available and binary

trait_row = None  # Specific key for Type_1_Diabetes_Risk is not explicitly identified
age_row = 1
gender_row = 4

# Define data type conversion functions

def convert_trait(value):
    # Trait data is not directly available; assuming inferred logic is not reliable without concrete evidence
    return None

def convert_age(value):
    # Extracts the numeric value from the string "age at sample (months): value"
    try:
        return float(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    # Maps gender to binary (0 for female, 1 for male)
    try:
        gender = value.split(': ')[1].strip().lower()
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
        else:
            return None
    except:
        return None

# Save cohort information
save_cohort_info('GSE30209', './preprocessed/Type_1_Diabetes_Risk/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Type_1_Diabetes_Risk', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Type_1_Diabetes_Risk/trait_data/GSE30209.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
